In [68]:
from qiskit.optimization import QuadraticProgram
from qiskit.aqua import aqua_globals
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer

from qiskit.optimization.algorithms import SlsqpOptimizer
from qiskit.optimization.algorithms import GroverOptimizer
import random

In [82]:
problem = QuadraticProgram()
# specify problem here

binary_vars=7
int_vars=0
continuous_vars=3
variables=[]
var_values=[]
size=binary_vars
for i in range(size):
    variables.append(problem.binary_var(name='b_%i'%i))
    var_values.append(random.randint(0,1))
size=int_vars
for i in range(size):
    low_b=random.randint(0,5)
    up_b=random.randint(50,100)
    variables.append(problem.integer_var(low_b,up_b,name='i_%i'%i))
    var_values.append(random.randint(low_b,up_b))
size=continuous_vars
for i in range(size):
    low_b=random.randint(0,5)
    up_b=random.randint(50,100)
    variables.append(problem.continuous_var(low_b,up_b,name='f_%i'%i))
    var_values.append(random.randint(low_b,up_b))

num_constraints=int(problem.get_num_vars()*.6)
for i in range(num_constraints):
    #indices_b = random.choices(range(problem.get_num_binary_vars()), k=(int)(problem.get_num_binary_vars()*.4))
    indices = random.choices(range(problem.get_num_vars()), k=(int)(problem.get_num_vars()*.3))
    cdict = {}
    rhs=0
    for j in indices:
        coeff = random.randint(-10,20)
        cdict[variables[j].name]=coeff
        rhs=rhs + coeff*var_values[j]
    #print(cdict, rhs)        
    problem.linear_constraint(linear=cdict, sense='<=',rhs=rhs, name='lin_eq_%i'%i)

constant=random.randint(-10,20)
cdict = {}
for i in range(problem.get_num_vars()):
    coeff = random.randint(-10,20)
    cdict[variables[i].name]=coeff
print(cdict, rhs)        
problem.minimize(constant=constant, linear=cdict)
    

"""
b1 = problem.binary_var(name='b1')
i1 = problem.integer_var(name='i1')
x = problem.continuous_var(name="x")
y = problem.continuous_var(name="y")
problem.minimize(constant=3, linear={'b1': 1, 'i1':2}, quadratic={('y', 'y'): 2, ('x', 'x'): 1})
problem.linear_constraint(linear={'b1': 1, 'i1': 2}, sense='==', rhs=11, name='lin_eq')
problem.linear_constraint(linear={'x': 1, 'y': 2}, sense='<=', rhs=30, name='lin_leq')
problem.linear_constraint(linear={'x': 2, 'i1':1}, sense='>=', rhs=10, name='lin_leq2')
problem.linear_constraint(linear={'y': 2, 'b1':1}, sense='>=', rhs=3, name='lin_leq3')
problem.quadratic_constraint(linear={'i1': 1, 'b1': 1}, quadratic={('y', 'y'): 1, ('x', 'x'): 1}, sense='<=', rhs=180, name='quad_leq')
"""    
#1 5 8 10
optimizer = SlsqpOptimizer()


def solve(optimizer, qubo, log=True):
    import logging
    import time
    from qiskit.aqua import set_qiskit_aqua_logging
    
    if qubo.get_num_vars() >10:
        aqua_globals.massive=True
        
    if log==True:
        set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL

    startTime=time.time()
    result=optimizer.solve(qubo)
    runTime=time.time()-startTime
    set_qiskit_aqua_logging(logging.NOTSET)  # OFF

    return result, runTime


{'b_0': -4, 'b_1': 19, 'b_2': 6, 'b_3': 5, 'b_4': -4, 'b_5': 12, 'b_6': 19, 'f_0': 0, 'f_1': 0, 'f_2': -5} 20


In [83]:
from docplex.mp.constants import int_probtype_to_string
filename='..\\problemFiles\\qp-generated-'
varText=''
if binary_vars>0:
    varText='B%03d'%(binary_vars)+'-'
if int_vars>0:    
    varText=varText+'I%03d'%(int_vars)+'-'
if continuous_vars>0:    
    varText=varText+'C%03d'%(continuous_vars)+'-'
m = problem.to_docplex()
cpx = m.get_cplex(do_raise=False)
if cpx:
    cpx_probtype = cpx.get_problem_type()
    filename=filename+int_probtype_to_string(cpx_probtype)+'-'

filename=filename+varText+str(random.randint(122100,999999))
problem.write_to_lp_file(filename)

In [77]:
print(problem.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 19 b_0 + 13 b_1 + 12 b_2 - 6 b_3 - 6 b_4 + 2 f_0 - 2 f_1 -7
Subject To
 lin_eq_0: - 8 b_0 <= 0
 lin_eq_1: - b_2 + 3 b_3 <= 0
 lin_eq_2: 15 b_2 + 11 b_4 <= 0
 lin_eq_3: 19 b_0 + 18 f_1 <= 954

Bounds
 0 <= b_0 <= 1
 0 <= b_1 <= 1
 0 <= b_2 <= 1
 0 <= b_3 <= 1
 0 <= b_4 <= 1
 5 <= f_0 <= 54
       f_1 <= 91

Binaries
 b_0 b_1 b_2 b_3 b_4
End



In [19]:
result = solve(optimizer, problem)

QiskitOptimizationError: 'Incompatible problem: The SLSQP optimizer supports only continuous variables'

In [10]:
print(result)

(optimal function value: 2.0
optimal value: [2. 1.]
status: SUCCESS, 0.030984163284301758)


In [78]:
num_qubits=print(op.num_qubits)

9


In [79]:
#Sometimes a QuadraticProgram might also directly be given in the form of an Operator. 
#For such cases, Qiskit also provides a converter from an Operator back to a QuadraticProgram, 
#which we illustrate in the following.

qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 1151 x_0 - 1119 x_1 - 1117 x_2 + 202733 x_3 - 216190 x_4 - 1168 x_5
      - 122619 x_6 - 1145 x_7 + 262665 x_8 + [ 512 x_0*x_1 + 512 x_0*x_2
      + 512 x_0*x_3 + 512 x_0*x_4 + 512 x_0*x_5 + 512 x_0*x_6 + 512 x_0*x_7
      + 512 x_0*x_8 + 512 x_1*x_2 + 512 x_1*x_3 + 512 x_1*x_4 + 512 x_1*x_5
      + 512 x_1*x_6 + 512 x_1*x_7 + 512 x_1*x_8 + 512 x_2*x_3 + 512 x_2*x_4
      + 512 x_2*x_5 + 512 x_2*x_6 + 512 x_2*x_7 + 512 x_2*x_8 - 122368 x_3*x_4
      + 512 x_3*x_5 - 52736 x_3*x_6 + 512 x_3*x_7 + 439808 x_3*x_8 + 512 x_4*x_5
      + 200192 x_4*x_6 + 512 x_4*x_7 + 246272 x_4*x_8 + 512 x_5*x_6
      + 512 x_5*x_7 + 512 x_5*x_8 + 512 x_6*x_7 + 107008 x_6*x_8 + 512 x_7*x_8
      ]/2 + 239872
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8
E

In [4]:
qubo=mod

In [5]:
#This converter allows to translate an Operator to a QuadraticProgram and 
#then solve the problem with other algorithms that are not based on the Ising Hamiltonian representation, 
#such as the GroverOptimizer.

In [6]:
print(qubo.get_num_vars())

3


In [7]:
backend=get_best_backend(backend='ibmq_manhattan',apiToken=MY_API_TOKEN)

E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>, <AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>]
<AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>


In [8]:
backend=get_best_backend(backend='ibmq_montreal',apiToken=MY_API_TOKEN)

E:\DevelopmentEnvironments\Anaconda3\envs\qiskit_env\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:109: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>, <AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>]
<AccountProvider for IBMQ(hub='ibm-q-internal', group='deployed', project='default')>


## Solving a QUBO with the MinimumEigenOptimizer

In [9]:
#We start by initializing the MinimumEigensolver we want to use.
#backend=BasicAer.get_backend('qasm_simulator')
aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(backend, 
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed,
                                   optimization_level=3,
                                  skip_qobj_validation=False)

vqe_core = VQE(quantum_instance=quantum_instance, max_evals_grouped=101)

qaoa_core = QAOA(quantum_instance=quantum_instance, max_evals_grouped=101)
#qaoa_core = QAOA(quantum_instance=backend)

# Create a Grover Optimizer - it will terminate after there have been 10 iterations of GAS without progress (i.e. the value of y does not change). 
grover_core = GroverOptimizer(qubo.get_num_vars(), num_iterations=5, quantum_instance=backend) #, max_evals_grouped=101)

exact_core = NumPyMinimumEigensolver()

#Then, we use the MinimumEigensolver to create MinimumEigenOptimizer.
qaoa = MinimumEigenOptimizer(qaoa_core)    # using QAOA
vqe = MinimumEigenOptimizer(vqe_core)      # using VQE
#grover = MinimumEigenOptimizer(grover_core)# using Grover's
exact = MinimumEigenOptimizer(exact_core)  # using the exact classical numpy minimum eigen solver

In [10]:
#We first use the MinimumEigenOptimizer based on the classical exact 
#NumPyMinimumEigensolver to get the optimal benchmark solution for this small example.
exact_result, runTime = solve(exact, qubo)
print(exact_result)
print('Time Taken:', runTime)

KeyboardInterrupt: 

In [109]:
#Next we apply the MinimumEigenOptimizer based on QAOA to the same problem.
qaoa_result, qrunTime = solve(qaoa, qubo)
#qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

2020-12-28 17:32:05,733:qiskit.aqua.algorithms.vq_algorithm:INFO: Starting optimizer.
bounds=[(0, 3.141592653589793), (0, 6.283185307179586)]
initial point=[0, 0]
2020-12-28 17:32:16,050:qiskit.aqua.quantum_instance:DEBUG: ==== Before transpiler ====
2020-12-28 17:32:16,050:qiskit.aqua.quantum_instance:DEBUG: Submitting 1 circuits.
0-th circuit: 12 qubits, 12 classical bits and 129 operations with depth 43
op_counts: OrderedDict([('cx', 54), ('rz', 39), ('h', 12), ('rx', 12), ('measure', 12)])

2020-12-28 17:32:16,050:qiskit.aqua.quantum_instance:DEBUG: ====  After transpiler ====
2020-12-28 17:32:16,050:qiskit.aqua.quantum_instance:DEBUG: Submitting 1 circuits.
0-th circuit: 65 qubits, 12 classical bits and 345 operations with depth 121
op_counts: OrderedDict([('cx', 232), ('u2', 46), ('u1', 39), ('u3', 16), ('measure', 12), ('barrier', 1)])

2020-12-28 17:32:16,068:qiskit.aqua.operators.converters.circuit_sampler:DEBUG: Parameter binding 10.00547 (ms)
2020-12-28 17:32:20,476:qiskit.a

optimal function value: 220.0
optimal value: [0. 0. 1. 1. 0. 0. 0. 1. 0.]
status: SUCCESS


In [86]:
print('Time Taken:', qrunTime)

Time Taken: 207.5039405822754


In [87]:
#Next we apply the MinimumEigenOptimizer based on VQE to the same problem.
vqe_result, vqerunTime = solve(vqe, qubo)
print(vqe_result)

2020-12-27 16:37:31,017:qiskit.aqua.algorithms.vq_algorithm:INFO: Starting optimizer.
bounds=[(-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.141592653589793, 3.141592653589793), (-3.

optimal function value: -1.0
optimal value: [1. 0. 0. 0. 1. 1. 1. 1. 0.]
status: SUCCESS


In [88]:
print(vqerunTime)

2697.744309425354


## RecursiveMinimumEigenOptimizer

In [79]:
#The RecursiveMinimumEigenOptimizer takes a MinimumEigenOptimizer as input and 
#applies the recursive optimization scheme to reduce the size of the problem one variable at a time. 
#Once the size of the generated intermediate problem is below a given threshold (min_num_vars), 
#the RecursiveMinimumEigenOptimizer uses another solver (min_num_vars_optimizer), 
#e.g., an exact classical solver such as CPLEX or the MinimumEigenOptimizer based on the NumPyMinimumEigensolver.

#In the following, we show how to use the RecursiveMinimumEigenOptimizer 
#using the two MinimumEigenOptimizer introduced before.

#First, we construct the RecursiveMinimumEigenOptimizer 
#such that it reduces the problem size from 3 variables to 1 variable and 
#then uses the exact solver for the last variable. 
#Then we call solve to optimize the considered problem.

rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=5, min_num_vars_optimizer=exact)

rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)



optimal function value: 4159.0
optimal value: [0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
status: SUCCESS


In [ ]:
# Solve using Grover's Optimizer
grover_result, groverRunTime = solve(grover_core, qubo)
print(grover_result)

In [ ]:
print(groverRunTime)

In [ ]:
import logging
import time
from qiskit.aqua import set_qiskit_aqua_logging

if qubo.get_num_vars() >10:
    aqua_globals.massive=True
set_qiskit_aqua_logging(logging.DEBUG)  # levels NOTSET<DEBUG<INFO<WARNING<ERROR<CRITICAL

startTime=time.time()
result = grover_core.solve(qubo)
runTime=time.time()-startTime
#set_qiskit_aqua_logging(logging.NOTSET)  # OFF
print("x={}".format(result.x))
print("fval={}".format(result.fval))
print('Runtime: ', runTime)

2020-12-30 13:42:00,214:qiskit.aqua.utils.run_circuits:INFO: Backend status: <qiskit.providers.models.backendstatus.BackendStatus object at 0x000001C01C24BEC8>
2020-12-30 13:42:00,215:qiskit.aqua.utils.run_circuits:INFO: There are 1 jobs are submitted.
2020-12-30 13:42:00,216:qiskit.aqua.utils.run_circuits:INFO: All job ids:
['5fec3649e60331001b30981f']
2020-12-30 13:42:00,216:qiskit.aqua.utils.run_circuits:INFO: Running 0-th qobj, job id: 5fec3649e60331001b30981f
2020-12-30 13:42:01,173:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f, status: JobStatus.VALIDATING
2020-12-30 13:42:07,251:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f is queued at position 1
2020-12-30 13:42:12,759:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f is queued at position 1
2020-12-30 13:42:18,834:qiskit.aqua.utils.run_circuits:INFO: Job id: 5fec3649e60331001b30981f is queued at position 1
2020-12-30 13:42:24,876:qiskit.aqua.utils.run_circuit